In [1]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [2]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

# בדיקות ל־OOP: מחלקות, ירושה, פולימורפיזם ו־patch בסיסי

במחברת זו נלמד לבדוק קוד מונחה־עצמים (OOP) בעזרת `unittest`:
- איך בודקים **מצב פנימי (state)** של אובייקט ושינויים בעקבות קריאות למתודות.
- איך בודקים **ירושה ופולימורפיזם**: מחלקת בסיס ומחלקות נגזרות עם אותו ממשק.
- איך בודקים **חריגות** במחלקות.
- **מבוא עדין** ל־`unittest.mock.patch` כדי לנתק תלות חיצונית קלה (למשל זמן ריצה).

> הערה: להרצה בתוך Jupyter השתמשו ב־`unittest.main(argv=[''], exit=False)` כדי לא לסגור את הקרנל.

## מטרות הלמידה
- לכתוב בדיקות עבור מחלקות: יצירה, מצב התחלתי, שינוי מצב, ולוגיקת מתודות.
- לעצב בדיקות שמוודאות התנהגות עקבית במחלקות יורשות (פולימורפיזם).
- להשתמש ב־`assert`-ים מתאימים: `assertEqual`, `assertTrue/False`, `assertRaises`, `assertAlmostEqual`.
- להשתמש ב־`unittest.mock.patch` באופן **מינימלי ופשוט** (ללא תכונות מתקדמות).

## 1) חימום: בדיקת מחלקה פשוטה עם מצב פנימי
בדוגמה זו נבדקת המחלקה `Counter`, שמנהלת ערך פנימי `_value` ומאפשרת להגדיל, להקטין, ולאפס אותו.  
המחלקה כוללת גם ולידציה (בדיקת תקינות קלט) — לא ניתן להגדיל או להקטין בערך שלילי.

מחלקת הבדיקות `TestCounter` מדגימה כיצד לבדוק **מצב פנימי משתנה** לאורך זמן בעזרת `unittest`:

1. **setUp()** — יוצרת אובייקט חדש של `Counter` לפני כל בדיקה, כדי שכל בדיקה תתחיל ממצב נקי.
2. **test_initial_state** — בודקת שהערך ההתחלתי הוא אפס.
3. **test_increment_and_decrement** — בודקת שהמתודות `increment` ו־`decrement` משנות את הערך כראוי.
4. **test_reset** — מוודאת שקריאה ל־`reset()` מאפסת את הערך חזרה ל־0.
5. **test_negative_step_raises** — בודקת שקריאה עם ערך שלילי (`step < 0`) מעלה חריגה (`ValueError`).

> זוהי דוגמה קלאסית לבדיקה של מחלקה עם **מצב פנימי (stateful object)**:  
> אנחנו בודקים גם ערכים תקינים וגם מקרי שגיאה, ומוודאים שה־state מתעדכן בהתאם.

In [3]:
import unittest

class Counter:
    def __init__(self):
        self._value = 0

    def increment(self, step=1):
        if step < 0:
            raise ValueError("step must be non-negative")
        self._value += step

    def decrement(self, step=1):
        if step < 0:
            raise ValueError("step must be non-negative")
        self._value -= step

    def reset(self):
        self._value = 0

    @property
    def value(self):
        return self._value
    
class TestCounter(unittest.TestCase):
    def setUp(self):
        """TODO: Create a new Counter instance before each test."""
        # TODO: initialize self.c as a new Counter object
        pass  # remove this line after completing the setup

    def test_initial_state(self):
        """TODO: Verify that a new counter starts at 0."""
        # TODO: use assertEqual to check self.c.value == 0
        pass # remove after completing the test

    def test_increment_and_decrement(self):
        """TODO: Test increment and decrement behavior."""
        # Arrange
        # TODO: start from a fresh counter (already done by setUp)
        # Act
        # TODO: call increment(3)
        # TODO: verify the value increased properly
        # TODO: call decrement(1)
        # TODO: verify the value decreased properly
        pass # remove after completing the test

    def test_reset(self):
        """TODO: Verify that reset() sets the counter back to 0."""
        # TODO: increment the counter (e.g., by 10)
        # TODO: call reset()
        # TODO: use assertEqual to check the value is 0
        pass # remove after completing the test

    def test_negative_step_raises(self):
        """TODO: Ensure that negative steps raise ValueError."""
        # TODO: use assertRaises(ValueError) for increment(-1)
        # TODO: use assertRaises(ValueError) for decrement(-2)
        pass # remove after completing the test

# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestCounter)
unittest.TextTestRunner(verbosity=2).run(suite)

test_increment_and_decrement (__main__.TestCounter.test_increment_and_decrement)
TODO: Test increment and decrement behavior. ... 

ok


test_initial_state (__main__.TestCounter.test_initial_state)
TODO: Verify that a new counter starts at 0. ... 

ok


test_negative_step_raises (__main__.TestCounter.test_negative_step_raises)
TODO: Ensure that negative steps raise ValueError. ... 

ok


test_reset (__main__.TestCounter.test_reset)
TODO: Verify that reset() sets the counter back to 0. ... 

ok


----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

`````{admonition} פתרון
:class: dropdown, tip
```python
class TestCounter(unittest.TestCase):
    def setUp(self):
        self.c = Counter()

    def test_initial_state(self):
        self.assertEqual(self.c.value, 0)

    def test_increment_and_decrement(self):
        self.c.increment(3)
        self.assertEqual(self.c.value, 3)
        self.c.decrement(1)
        self.assertEqual(self.c.value, 2)

    def test_reset(self):
        self.c.increment(10)
        self.c.reset()
        self.assertEqual(self.c.value, 0)

    def test_negative_step_raises(self):
        with self.assertRaises(ValueError):
            self.c.increment(-1)
        with self.assertRaises(ValueError):
            self.c.decrement(-2)
```
`````

```{note}
כאן בדקנו יצירת אובייקט, שינויי מצב (state) לאורך הזמן, וחריגות על קלט לא חוקי.
```

## 2) ירושה ופולימורפיזם: Sensor → TempSensor / PressureSensor
בדוגמה זו אנו רואים מבנה מונחה־עצמים (OOP) של מחלקת בסיס `Sensor` ושתי מחלקות יורשות – `TempSensor` ו־`PressureSensor`.

### המחלקות
- **Sensor (מחלקת בסיס):**  
  מגדירה מתודה כללית `read()` אך לא מממשת אותה — במקום זאת היא מעלה `NotImplementedError`.  
  כך נבטיח שכל מחלקה יורשת תצטרך לממש את המתודה בעצמה.

- **TempSensor (מד טמפרטורה):**  
  מקבלת טמפרטורה בצלזיוס, ומחזירה את הערך בקלווין לפי \( K = C + 273.15 \).  
  אם הערך נמוך מ־0 קלווין (כלומר מתחת לאפס המוחלט), המתודה תזרוק חריגה (`ValueError`).

- **PressureSensor (מד לחץ):**  
  מקבלת ערך בלחץ פסקל, ומחזירה אותו כל עוד הוא לא שלילי.  
  במקרה של ערך שלילי — נזרקת חריגה (`ValueError`).

In [4]:
import unittest

class Sensor:
    def read(self):
        """Return a numeric reading. Must be implemented by subclasses."""
        raise NotImplementedError("Subclasses must implement read()")

class TempSensor(Sensor):
    def __init__(self, celsius):
        self.celsius = float(celsius)

    def read(self):
        # Example: return temperature in Kelvin
        k = self.celsius + 273.15
        if k < 0:
            raise ValueError("Physical error: below 0 K")
        return k

class PressureSensor(Sensor):
    def __init__(self, pascal):
        self.pascal = float(pascal)

    def read(self):
        # Example: sanity check for negative pressure
        if self.pascal < 0:
            raise ValueError("Pressure cannot be negative")
        return self.pascal

### מחלקת הבדיקות – TestSensors
הבדיקות מדגימות **פולימורפיזם**: כל מחלקה יורשת את אותה מתודה `read()`,  
אבל מממשת אותה בצורה שונה.

1. **test_polymorphic_read**  
   עובר על רשימת חיישנים (`TempSensor`, `PressureSensor`) ומוודא שלכל אחד יש את אותה ממשק (`read`) שמחזיר ערך מסוג float.

2. **test_temp_sensor_kelvin**  
   בודקת שהמרת °C → K מתבצעת כראוי (0°C = 273.15K).

3. **test_pressure_sensor_positive**  
   בודקת שמד הלחץ מחזיר את הערך הנכון לערכים חיוביים.

4. **test_temp_sensor_below_abs_zero_raises**  
   מוודאת שטמפרטורה מתחת ל־0 קלווין גורמת ל־`ValueError`.

5. **test_pressure_sensor_negative_raises**  
   מוודאת שלחץ שלילי גם הוא גורם ל־`ValueError`.

In [5]:
# --- TODO: Complete the test suite ---
class TestSensors(unittest.TestCase):
    def test_polymorphic_read(self):
        """TODO: Verify that all sensors share the same interface (read)."""
        # TODO: create a list with a TempSensor and a PressureSensor
        # sensors = [...]
        # TODO: loop through sensors and assert that read() returns a float
        pass # remove after completing the test

    def test_temp_sensor_kelvin(self):
        """TODO: Check that 0°C equals 273.15K."""
        # TODO: create a TempSensor with 0°C
        # TODO: call read() and use assertAlmostEqual to check 273.15
        pass # remove after completing the test

    def test_pressure_sensor_positive(self):
        """TODO: Check that PressureSensor returns correct positive values."""
        # TODO: create a PressureSensor with a positive value (e.g., 500)
        # TODO: call read() and assertAlmostEqual to expected result
        pass # remove after completing the test

    def test_temp_sensor_below_abs_zero_raises(self):
        """TODO: Verify that temperature below absolute zero raises ValueError."""
        # TODO: create a TempSensor(-274)
        # TODO: use assertRaises(ValueError) when calling read()
        pass # remove after completing the test

    def test_pressure_sensor_negative_raises(self):
        """TODO: Verify that negative pressure raises ValueError."""
        # TODO: create a PressureSensor(-1)
        # TODO: use assertRaises(ValueError) when calling read()
        pass # remove after completing the test


# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestSensors)
unittest.TextTestRunner(verbosity=2).run(suite)

test_polymorphic_read (__main__.TestSensors.test_polymorphic_read)
TODO: Verify that all sensors share the same interface (read). ... 

ok


test_pressure_sensor_negative_raises (__main__.TestSensors.test_pressure_sensor_negative_raises)
TODO: Verify that negative pressure raises ValueError. ... 

ok


test_pressure_sensor_positive (__main__.TestSensors.test_pressure_sensor_positive)
TODO: Check that PressureSensor returns correct positive values. ... 

...

ok


test_temp_sensor_kelvin (__main__.TestSensors.test_temp_sensor_kelvin)
TODO: Check that 0°C equals 273.15K. ... 

ok


----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

`````{admonition} פתרון
:class: dropdown, tip
```python
class TestSensors(unittest.TestCase):
    def test_polymorphic_read(self):
        sensors = [
            TempSensor(25.0),     # ~298.15 K
            PressureSensor(101325)  # 1 atm in Pa
        ]
        for s in sensors:
            val = s.read()  # same interface
            self.assertTrue(isinstance(val, float))

    def test_temp_sensor_kelvin(self):
        t = TempSensor(0.0)
        self.assertAlmostEqual(t.read(), 273.15, places=6)

    def test_pressure_sensor_positive(self):
        p = PressureSensor(500.0)
        self.assertAlmostEqual(p.read(), 500.0, places=6)

    def test_temp_sensor_below_abs_zero_raises(self):
        t = TempSensor(-274.0)
        with self.assertRaises(ValueError):
            t.read()

    def test_pressure_sensor_negative_raises(self):
        p = PressureSensor(-1.0)
        with self.assertRaises(ValueError):
            p.read()

```
`````

```{note}
הבדיקות מוודאות שכל המחלקות הגזורות משתפות **ממשק זהה** (פולימורפיזם),  
ושמירה על חוקיות פיזיקלית מתקיימת באמצעות חריגות.
שבה הבדיקות מוודאות שכל מחלקה שומרת על אותו API (`read`) אך מיישמת אותו לפי ההיגיון הפיזיקלי שלה.
```

## 3) שימוש זהיר ב־patch: בדיקת מחלקה שתלוייה בזמן
בדוגמה זו נבדקת המחלקה `Stopwatch`, המדמה שעון עצר (סטופר).  
המחלקה מודדת זמן מצטבר באמצעות קריאות ל־`time.time()` מהמודול `time`,  
אך בבדיקות אנחנו לא רוצים להיות תלויים בשעון אמיתי —  
לכן נשתמש ב־`unittest.mock.patch` כדי **לזייף את הזמן**.

### מבנה המחלקה
- `start()` — מתחילה מדידה. אם הסטופר כבר פועל, תיזרק חריגה (`RuntimeError`).
- `stop()` — עוצרת מדידה ומוסיפה את פרק הזמן שעבר ל־`_elapsed`.
- `reset()` — מאפסת את כל הנתונים.
- `elapsed()` — מחזירה את הזמן שחלף עד כה (כולל מדידה נוכחית אם הסטופר פעיל).

In [6]:
import time
from unittest.mock import patch

class Stopwatch:
    def __init__(self):
        self._start = None
        self._elapsed = 0.0

    def start(self):
        if self._start is not None:
            raise RuntimeError("Already started")
        self._start = time.time()

    def stop(self):
        if self._start is None:
            raise RuntimeError("Not running")
        now = time.time()
        self._elapsed += now - self._start
        self._start = None

    def reset(self):
        self._start = None
        self._elapsed = 0.0

    def elapsed(self):
        if self._start is None:
            return self._elapsed
        return self._elapsed + (time.time() - self._start)

### מחלקת הבדיקות
1. **test_start_stop_elapsed_with_patch**  
   בעזרת `patch("time.time", side_effect=[...])` אנו מחליפים זמנית את הפונקציה `time.time`  
   כך שתחזיר רצף ערכים קבוע מראש (1000 → 1005 → 1010).  
   זה מאפשר לבדוק את חישוב ההפרשים בדיוק ובאופן דטרמיניסטי —  
   בלי קשר לשעון המחשב בפועל.

2. **test_double_start_raises**  
   בודקת שקריאה שנייה ל־`start()` בזמן שהסטופר כבר פועל גורמת ל־`RuntimeError`.

3. **test_stop_without_start_raises**  
   מוודאת שקריאה ל־`stop()` בלי התחלה קודמת גורמת גם היא ל־`RuntimeError`.


In [7]:
# --- TODO: Complete the test suite ---
class TestStopwatch(unittest.TestCase):
    def test_start_stop_elapsed_with_patch(self):
        """TODO: Simulate time flow using patch and test elapsed time."""
        # TODO: use patch("time.time", side_effect=[1000.0, 1005.0, 1010.0])
        # to simulate time progressing in fixed steps
        # Inside the patch:
        #   1. create a Stopwatch instance
        #   2. call start()  → should use 1000.0
        #   3. call stop()   → should use 1005.0 (elapsed = 5.0)
        #   4. use assertAlmostEqual to check that elapsed() == 5.0
        #   5. call start() again to verify restart behavior (uses 1010.0)
        # Outside the patch:
        #   assertAlmostEqual that sw._elapsed == 5.0
        pass  # remove after completing the test

    def test_double_start_raises(self):
        """TODO: Ensure that starting twice without stopping raises RuntimeError."""
        # TODO: use patch("time.time", return_value=1000.0)
        # TODO: create Stopwatch instance and call start()
        # TODO: call start() again inside assertRaises(RuntimeError)
        pass # remove after completing the test

    def test_stop_without_start_raises(self):
        """TODO: Ensure that calling stop() before start() raises RuntimeError."""
        # TODO: create Stopwatch instance
        # TODO: call stop() inside assertRaises(RuntimeError)
        pass # remove after completing the test

# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestStopwatch)
unittest.TextTestRunner(verbosity=2).run(suite)

test_double_start_raises (__main__.TestStopwatch.test_double_start_raises)
TODO: Ensure that starting twice without stopping raises RuntimeError. ... 

ok


test_start_stop_elapsed_with_patch (__main__.TestStopwatch.test_start_stop_elapsed_with_patch)
TODO: Simulate time flow using patch and test elapsed time. ... 

ok


test_stop_without_start_raises (__main__.TestStopwatch.test_stop_without_start_raises)
TODO: Ensure that calling stop() before start() raises RuntimeError. ... 

ok


----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

`````{admonition} פתרון
:class: dropdown, tip
```python
class TestStopwatch(unittest.TestCase):
    def test_start_stop_elapsed_with_patch(self):
        # Fake timeline: 1000 -> 1005 -> 1010 seconds
        with patch("time.time", side_effect=[1000.0, 1005.0, 1010.0]):
            sw = Stopwatch()
            sw.start()    # uses 1000.0
            sw.stop()     # uses 1005.0 → elapsed += 5.0
            self.assertAlmostEqual(sw.elapsed(), 5.0, places=6)

            sw.start()    # uses 1010.0
            # no stop yet; elapsed() will read "current" time next if called
        # Outside the patch the real clock is back; to keep test deterministic,
        # assert only the part that happened under patch:
        self.assertAlmostEqual(sw._elapsed, 5.0, places=6)

    def test_double_start_raises(self):
        with patch("time.time", return_value=1000.0):
            sw = Stopwatch()
            sw.start()
            with self.assertRaises(RuntimeError):
                sw.start()

    def test_stop_without_start_raises(self):
        sw = Stopwatch()
        with self.assertRaises(RuntimeError):
            sw.stop()

```
`````


```{note}
השתמשנו ב־`patch("time.time", side_effect=[...])` כדי לשלוט ברצף הזמנים.  
כך הבדיקה **דטרמיניסטית**, מהירה, וסגורה לתלות חיצונית.
בשורה התחתונה — זו דוגמה מצוינת לשימוש ב־**mocking** כדי לבדוק קוד שתלוי בזמן אמיתי  
בצורה מבוקרת, מהירה, וללא תלות בתזמון מערכת אמיתי.
```

## 4) תרגול עצמי — מחלקת MeasurementLogger 

נרצה לבדוק לוגיקה של **שמירת מדידות ניסוי** במבנה פנימי,  
כאילו מדובר ביומן מדידות של ניסוי פיזיקלי (למשל מדידת טמפרטורות או זמני נפילה).  

מטרת התרגיל: לתרגל דפוס *Arrange–Act–Assert*, בדיקת **state פנימי**, וחריגות על קלט שגוי.

**תיאור כללי של המחלקה:**
- `MeasurementLogger()` מייצגת רושם ניסוי פשוט.
- `record(value)` — מוסיפה מדידה חדשה (מספר ממשי). אם הקלט אינו מספר → `TypeError`.  
  אם הערך שלילי (לא פיזיקלי במקרה מסוים) → `ValueError`.
- `history()` — מחזירה עותק של רשימת כל המדידות שנרשמו.
- `clear()` — מאפסת את ההיסטוריה.

### MeasurementLogger
המחלקה `MeasurementLogger` מדמה רושם ניסוי — אובייקט ששומר מדידות פיזיקליות (כמו טמפרטורה, זמן או מתח).  
המטרה היא לוודא שהמדידות חוקיות (לא שליליות, ולא טקסט), ולאפשר צפייה בהיסטוריית המדידות מבלי לפגוע במידע המקורי.

- **record(value)** — מוסיפה מדידה חדשה רק אם היא תקינה:
  - הערך חייב להיות מסוג `int` או `float`, אחרת נזרקת חריגה `TypeError`.  
  - הערך חייב להיות לא שלילי, אחרת נזרקת `ValueError`.  
- **history()** — מחזירה **עותק** של רשימת המדידות, כדי שלא ניתן יהיה לשנות את הנתונים המקוריים מבחוץ.  
- **clear()** — מוחקת את כל ההיסטוריה (מאפסת את רשימת המדידות).  

זהו תרגיל קלאסי להבנת **state פנימי**, בדיקות תקינות, והקפדה על בידוד מידע (encapsulation) —  
עקרונות חשובים מאוד במערכות ניסוי ומדידה פיזיקליות.

In [8]:
# --- TODO: Complete the Measurement Logger ---
class MeasurementLogger:
    def __init__(self):
        """TODO: Initialize internal storage for measurements."""
        # TODO: create an empty list to hold numeric measurement values
        pass # remove after completing the test

    def record(self, value):
        """TODO: Validate and record a new measurement."""
        # TODO: check that value is int or float; otherwise raise TypeError
        # TODO: check that value is non-negative; otherwise raise ValueError
        # TODO: append value to the internal list
        pass # remove after completing the test

    def history(self):
        """TODO: Return a COPY of the stored measurements."""
        # TODO: return a copy (not a reference) of the internal list
        pass # remove after completing the test

    def clear(self):
        """TODO: Clear all stored measurements."""
        # TODO: empty the internal list
        pass # remove after completing the test

`````{admonition} פתרון
:class: dropdown, tip
```python
class MeasurementLogger:
    def __init__(self):
        """Initialize internal storage for measurements."""
        self._measurements = []  # list to hold numeric measurement values

    def record(self, value):
        """Validate and record a new measurement."""
        # Check type
        if not isinstance(value, (int, float)):
            raise TypeError("Measurement must be a number (int or float)")
        # Check range
        if value < 0:
            raise ValueError("Measurement must be non-negative")
        # Store value
        self._measurements.append(float(value))

    def history(self):
        """Return a COPY of the stored measurements."""
        return self._measurements.copy()

    def clear(self):
        """Clear all stored measurements."""
        self._measurements.clear()
```
`````

### TestMeasurementLogger
בדוגמה זו אנחנו בודקים את המחלקה `MeasurementLogger`, ששומרת רשימת מדידות ניסוי.

מטרת הבדיקות:
1. **בדיקת שמירה וקריאה של מדידות (test_record_and_history)**  
   מוודאת שערכים שנרשמו באמצעות `record()` אכן מופיעים בהיסטוריה (`history()`).
2. **בדיקת עותק (test_history_is_copy)**  
   בודקת ש־`history()` מחזירה עותק של הרשימה, כך ששינוי חיצוני לא ישפיע על הנתונים הפנימיים.
3. **בדיקות חריגות (test_record_invalid_type_raises, test_record_negative_value_raises)**  
   בודקות שהמחלקה מגיבה נכון לקלטים לא חוקיים — טיפוס לא מספרי או ערך שלילי.

זוהי דוגמה מצוינת לבדוק **state פנימי** ולוודא שהמחלקה מתנהגת נכון תחת תנאים פיזיקליים מציאותיים (לדוגמה, מדידה לא יכולה להיות שלילית).


In [9]:
# --- TODO: Complete the Measurement Logger ---
class TestMeasurementLogger(unittest.TestCase):
    def setUp(self):
        """Prepare a fresh logger before each test."""
        self.log = MeasurementLogger()

    def test_record_and_history(self):
        """TODO: Check that recorded values appear in history()."""
        # Arrange
        measurements = [1.2, 3.4, 5.6]
        # Act
        # TODO: record each value in measurements
        # Assert
        # TODO: use assertEqual to verify history() == [1.2, 3.4, 5.6]
        pass # remove after completing the test

    def test_history_is_copy(self):
        """TODO: Ensure history() returns a copy, not the original list."""
        # Arrange
        # TODO: record a single value, then get history() into h
        # Act
        # TODO: modify h (e.g., h.append(999))
        # Assert
        # TODO: verify that calling history() again still returns the original data
        pass # remove after completing the test

    def test_record_invalid_type_raises(self):
        """TODO: Verify that non-numeric input raises TypeError."""
        # TODO: record("not a number") inside assertRaises(TypeError)
        pass # remove after completing the test

    def test_record_negative_value_raises(self):
        """TODO: Verify that negative measurement raises ValueError."""
        # TODO: record(-5) inside assertRaises(ValueError)
        pass # remove after completing the test

    def test_clear(self):
        """TODO: Verify that clear() empties the measurement list."""
        # Arrange
        # TODO: record a few measurements
        # Act
        # TODO: call clear()
        # Assert
        # TODO: verify history() returns an empty list
        pass # remove after completing the test

# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestMeasurementLogger)
unittest.TextTestRunner(verbosity=2).run(suite)

test_clear (__main__.TestMeasurementLogger.test_clear)
TODO: Verify that clear() empties the measurement list. ... 

ok


test_history_is_copy (__main__.TestMeasurementLogger.test_history_is_copy)
TODO: Ensure history() returns a copy, not the original list. ... 

ok


test_record_and_history (__main__.TestMeasurementLogger.test_record_and_history)
TODO: Check that recorded values appear in history(). ... 

...

ok


test_record_negative_value_raises (__main__.TestMeasurementLogger.test_record_negative_value_raises)
TODO: Verify that negative measurement raises ValueError. ... 

ok


----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

`````{admonition} פתרון
:class: dropdown, tip
```python
class TestMeasurementLogger(unittest.TestCase):
    def setUp(self):
        """Prepare a fresh logger before each test."""
        self.log = MeasurementLogger()

    def test_record_and_history(self):
        """Check that recorded values appear in history()."""
        # Arrange
        measurements = [1.2, 3.4, 5.6]
        # Act
        for m in measurements:
            self.log.record(m)
        # Assert
        self.assertEqual(self.log.history(), measurements)

    def test_history_is_copy(self):
        """Ensure history() returns a copy, not the original list."""
        # Arrange
        self.log.record(1.0)
        h = self.log.history()
        # Act
        h.append(999)  # modify the returned list
        # Assert
        # internal data should not be affected
        self.assertEqual(self.log.history(), [1.0])

    def test_record_invalid_type_raises(self):
        """Verify that non-numeric input raises TypeError."""
        with self.assertRaises(TypeError):
            self.log.record("not a number")

    def test_record_negative_value_raises(self):
        """Verify that negative measurement raises ValueError."""
        with self.assertRaises(ValueError):
            self.log.record(-5)

    def test_clear(self):
        """Verify that clear() empties the measurement list."""
        # Arrange
        for v in [1.0, 2.0, 3.0]:
            self.log.record(v)
        # Act
        self.log.clear()
        # Assert
        self.assertEqual(self.log.history(), [])
```
`````

## סיכום

במחברת זו:
- בדקנו מחלקות עם **state** משתנה וחריגות קלט.
- אימתנו **פולימורפיזם** במחלקות יורשות ושמרנו על ממשק אחיד.
- השתמשנו ב־`patch` כדי לבודד תלות חיצונית באופן **קל ונקי**.
- תרגלנו כתיבה של בדיקות OOP עם `subTest` ועם דגש על **AAA** ו־**Edge Cases**.

במחברת הבאה (חלק 4) נתמקד בבדיקות לקוד נומרי בפיזיקה: טולרנסים עשרוניים, רעש אקראי, ובדיקות המרות יחידות.
